In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
%pip install -q decorator==5.0.9
!pip install -q ipywidgets

import csv
import math
import re
import imp
import json
import numpy as np
import pandas as pd
import difflib as dl
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
import matplotlib.pyplot as plt
import networkx as nx
import seaborn as sns
from collections import Counter

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install -q cltk==1.0.22
%pip install -q strsim
%pip install -q leven

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [30]:
# importing local modules

import sys
sys.path.append('../src/')

import text_cleanup.text_cleanup as thesisCleanUp
import preprocessing.text_preprocessing as thesisTextPreprocessing
import data.reader as thesisDataReader
import utils.utils as thesisUtils
import features.tf_idf.n_gram as thesisTfIdfNgramFeatures
import features.count_vectorizer.n_gram as thesisCountVectorizerNgramFeatures
import similarities.cosine as thesisCosineSimilarities
import similarities.levenshtein as thesisLevenshteinSimilarities
import vocabulary.vocabulary as thesisVocabulary
import features.lexical as thesisLexicalFeatures
import similarities.cosine as thesisCosineSimilarity
import text_cleanup.text_cleanup as thesisTextCleanUp
import p_aligment.p_aligment as thesisPAligment
import features.model_features as thesisModelFeatures

imp.reload(thesisLexicalFeatures)
imp.reload(thesisCosineSimilarity)
imp.reload(thesisCleanUp)
imp.reload(thesisTextPreprocessing)
imp.reload(thesisDataReader)
imp.reload(thesisUtils)
imp.reload(thesisTfIdfNgramFeatures)
imp.reload(thesisVocabulary)
imp.reload(thesisCosineSimilarities)
imp.reload(thesisTextCleanUp)
imp.reload(thesisCountVectorizerNgramFeatures)
imp.reload(thesisPAligment)
imp.reload(thesisLevenshteinSimilarities)
imp.reload(thesisModelFeatures)

<module 'features.model_features' from '../src/features/model_features.py'>

# Create burchard version in original london format

1. create london/zwickau versions without words/letters replacement - replace only numbers, punctuation, etc...
2. create word mapping of a new created versions in 1 to regular processed versions
3. change words in burchard candidate according to mapping in 2 - it should return us burchard in original format

In [5]:
london_corpus_by_new_line_without_words_processing = thesisDataReader.get_london_by_new_line_without_words_processing()
zwickau_corpus_by_new_line_without_words_processing = thesisDataReader.get_zwickau_by_new_line_without_words_processing()

In [6]:
london_original_to_processed_mapping = thesisVocabulary.create_pre_post_processing_map(
    ' \n'.join(london_corpus_by_new_line_without_words_processing)
)

In [7]:
burchard_version_with_original_london_text = []

for p in thesisDataReader.get_burchard_candidate_version_based_on_strongly_similar_london_base():
    new_p = ''
    
    for word in p.split():
        original_word = london_original_to_processed_mapping[london_original_to_processed_mapping['after'] == word]['before'].values[0]
        new_p = f'{new_p} {original_word}' if len(new_p) > 0 else original_word
    
    burchard_version_with_original_london_text.append(new_p)

# TF_IDF

In [8]:
model_features_df = thesisModelFeatures.create_features_df(
    london_corpus_by_new_line_without_words_processing,
    zwickau_corpus_by_new_line_without_words_processing,
    None
#     burchard_version_with_original_london_text
)

In [31]:
tf_idf_results = thesisModelFeatures.run_models(model_features_df)

/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaco

/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result

In [32]:
tf_idf_results

,precision_macro,recall_macro,f1_macro,f1_micro,f1_weighted,accuracy
SVM_linear,0.251064,0.500000,0.334272,0.502128,0.335711,0.502128
SVM_RBF,0.236190,0.259058,0.242737,0.259309,0.242947,0.259309
DecisionTreeClassifier,0.536232,0.535417,0.533660,0.535372,0.533671,0.535372
GaussianProcessClassifier,0.251064,0.500000,0.334272,0.502128,0.335711,0.502128
RandomForestClassifier,0.428117,0.428804,0.427735,0.428901,0.427811,0.428901
MLPClassifier,0.324271,0.339221,0.326211,0.338697,0.325841,0.338697
GaussianNB,0.275304,0.280888,0.275933,0.280496,0.275684,0.280496
KNeighborsClassifier,0.402640,0.405797,0.401431,0.405718,0.401407,0.405718
AdaBoostClassifier,0.582353,0.581522,0.580046,0.581560,0.580082,0.581560
QuadraticDiscriminantAnalysis,0.425054,0.428261,0.423320,0.428635,0.423572,0.428635


In [10]:
tf_idf_results

,precision_macro,recall_macro,f1_macro,f1_micro,f1_weighted,accuracy
SVC_linear,0.327639,0.330435,0.328073,0.330363,0.328042,0.330363
DecisionTreeClassifier,0.508215,0.508243,0.507201,0.508333,0.507267,0.508333
RandomForestClassifier,0.379333,0.380707,0.378863,0.380541,0.378763,0.380541
GaussianNB,0.275304,0.280888,0.275933,0.280496,0.275684,0.280496


# td_idf + cosine_similarity

In [33]:
model_features_df = thesisModelFeatures.create_features_df(
    london_corpus_by_new_line_without_words_processing,
    zwickau_corpus_by_new_line_without_words_processing,
    None,
    features = {'tfidf', 'inner_mean_cosine_similarity_score'}
#     burchard_version_with_original_london_text
)

In [34]:
tf_idf_cosine_results = thesisModelFeatures.run_models(model_features_df)

/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaco

/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result

In [35]:
tf_idf_cosine_results

,precision_macro,recall_macro,f1_macro,f1_micro,f1_weighted,accuracy
SVM_linear,0.251064,0.500000,0.334272,0.502128,0.335711,0.502128
SVM_RBF,0.236190,0.259058,0.242737,0.259309,0.242947,0.259309
DecisionTreeClassifier,0.578307,0.577536,0.576426,0.577261,0.576323,0.577261
GaussianProcessClassifier,0.251064,0.500000,0.334272,0.502128,0.335711,0.502128
RandomForestClassifier,0.352192,0.353261,0.351971,0.353324,0.352034,0.353324
MLPClassifier,0.341810,0.351630,0.342828,0.351463,0.342736,0.351463
GaussianNB,0.275304,0.280888,0.275933,0.280496,0.275684,0.280496
KNeighborsClassifier,0.402640,0.405797,0.401431,0.405718,0.401407,0.405718
AdaBoostClassifier,0.584572,0.583605,0.581923,0.583644,0.581959,0.583644
QuadraticDiscriminantAnalysis,0.425054,0.428261,0.423320,0.428635,0.423572,0.428635


In [24]:
tf_idf_cosine_results

,precision_macro,recall_macro,f1_macro,f1_micro,f1_weighted,accuracy
SVC_linear,0.329648,0.332518,0.330106,0.332491,0.330106,0.332491
DecisionTreeClassifier,0.553309,0.552174,0.550387,0.552128,0.550385,0.552128
RandomForestClassifier,0.410750,0.412047,0.410148,0.411968,0.410119,0.411968
GaussianNB,0.275304,0.280888,0.275933,0.280496,0.275684,0.280496
KNeighborsClassifier,0.402640,0.405797,0.401431,0.405718,0.401407,0.405718


In [14]:
tf_idf_cosine_results

,precision_macro,recall_macro,f1_macro,f1_micro,f1_weighted,accuracy
SVC_linear,0.329648,0.332518,0.330106,0.332491,0.330106,0.332491
DecisionTreeClassifier,0.584459,0.583605,0.582557,0.583555,0.582557,0.583555
RandomForestClassifier,0.398383,0.399638,0.397779,0.399690,0.397828,0.399690
GaussianNB,0.275304,0.280888,0.275933,0.280496,0.275684,0.280496


In [19]:
thesisLevenshteinSimilarities.get_inner_version_best_similarities(london_corpus_by_new_line_without_words_processing)

[[(20, 2)],
 [(311, 311)],
 [(79, 124)],
 [(311, 252)],
 [(273, 220)],
 [(270, 402)],
 [(303, 252)],
 [(289, 211)],
 [(79, 138)],
 [(79, 161)],
 [(45, 78)],
 [(79, 137)],
 [(79, 49)],
 [(79, 70)],
 [(287, 53)],
 [(79, 38)],
 [(79, 88)],
 [(79, 95)],
 [(79, 118)],
 [(270, 109)],
 [(289, 116)],
 [(97, 147)],
 [(11, 27)],
 [(30, 165)],
 [(289, 130)],
 [(289, 439)],
 [(311, 346)],
 [(311, 198)],
 [(79, 314)],
 [(289, 91)],
 [(79, 84)],
 [(161, 253)],
 [(54, 98)],
 [(190, 95)],
 [(289, 111)],
 [(287, 99)],
 [(287, 81)],
 [(145, 221)],
 [(79, 194)],
 [(79, 154)],
 [(79, 111)],
 [(79, 85)],
 [(271, 253)],
 [(79, 128)],
 [(79, 316)],
 [(311, 127)],
 [(287, 233)],
 [(287, 63)],
 [(287, 35)],
 [(289, 88)],
 [(79, 219)],
 [(79, 337)],
 [(79, 169)],
 [(70, 110)],
 [(79, 102)],
 [(79, 340)],
 [(287, 83)],
 [(229, 80)],
 [(287, 33)],
 [(79, 195)],
 [(79, 184)],
 [(55, 52)],
 [(28, 76)],
 [(51, 45)],
 [(79, 108)],
 [(287, 103)],
 [(51, 58)],
 [(70, 91)],
 [(79, 67)],
 [(51, 161)],
 [(51, 74)],
 [(79,

In [21]:
thesisLevenshteinSimilarities.get_inner_version_best_similarities(london_corpus_by_new_line_without_words_processing)

[[(20, 0.4)],
 [(311, 39.50580808080808)],
 [(79, 16.65292207792208)],
 [(311, 32.79213564213566)],
 [(79, 29.744047619047635)],
 [(79, 54.06735209235206)],
 [(303, 34.09168331668334)],
 [(289, 28.854906204906218)],
 [(79, 19.6373015873016)],
 [(79, 23.00556110556111)],
 [(45, 12.728030303030303)],
 [(79, 18.349206349206355)],
 [(79, 7.058549783549784)],
 [(79, 8.870454545454544)],
 [(287, 7.692857142857141)],
 [(79, 5.155050505050505)],
 [(79, 13.145418470418472)],
 [(79, 15.412337662337663)],
 [(79, 16.68199855699856)],
 [(270, 14.203843378843379)],
 [(289, 14.877164502164504)],
 [(79, 20.197372072372087)],
 [(11, 4.367857142857143)],
 [(30, 23.204670329670346)],
 [(289, 15.87187812187812)],
 [(289, 59.354964479964465)],
 [(311, 49.19842934842932)],
 [(311, 24.978187090687104)],
 [(79, 45.75878843378841)],
 [(289, 13.777561327561324)],
 [(30, 11.184163059163057)],
 [(79, 36.45831668331668)],
 [(54, 13.900793650793654)],
 [(190, 11.613586413586411)],
 [(289, 12.452648740148739)],
 [(2

In [27]:
c = [('s', 1), ('v',2)]
     
for string, num in c:
    print(num)

1
2
